#初始化

Tiktoken是一个Python库，由OpenAI开发，用于计算一个给定文本字符串在特定的Transformer语言模型（例如GPT-3）中会使用多少个tokens。这个库可以帮助开发者更好地理解他们的文本输入将如何被语言模型处理，特别是在考虑到OpenAI API的使用成本时，这个库可以帮助开发者预估他们的API使用成本。

在Transformer模型中，文本被分解成tokens，这些tokens可以是一个字符，一个词，或者一个子词，这取决于模型的训练方式。在GPT-3中，一个token通常是一个字符或者一个子词。由于OpenAI的API计费是基于token数量的，所以理解你的文本输入将使用多少tokens是很重要的。

Faiss是一个由Facebook AI Research开发的库，用于高效地处理和搜索大规模的向量数据。在许多AI和机器学习应用中，我们经常需要处理高维度的向量数据，并需要快速地找到与给定向量最相似的向量。例如，在推荐系统中，我们可能需要找到与用户的兴趣向量最相似的商品向量；在自然语言处理中，我们可能需要找到与给定词向量最相似的词向量。这种操作被称为“最近邻搜索”。

In [ ]:
!pip install openai
!pip install langchain
!pip install faiss-cpu
!pip install tiktoken


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your key"

#1.Document Load 文件加载
创建了一个Hacker News加载器（HNLoader），并指定了要加载的Hacker News文章的URL
打印出前两个评论的前150个字符。

Hacker News是一个由Y Combinator创办的社区网站，用户可以在上面分享和讨论计算机科学和创业相关的新闻和资源。它的用户群体主要是技术人员和创业者，因此在Hacker News上可以找到很多高质量的技术和创业相关的内容。

Hacker News的文档加载器（HNLoader）是一个专门用来从Hacker News网站上加载数据的工具。它可以从Hacker News的帖子中提取出评论和其他相关信息。这对于需要使用Hacker News数据的研究和开发工作来说是非常有用的。

In [ ]:
from langchain.document_loaders import HNLoader
loader = HNLoader("https://news.ycombinator.com/item?id=34422627")

data = loader.load()
print (f"Found {len(data)} comments")
print (f"Here's a sample:\n\n{''.join([x.page_content[:150] for x in data[:2]])}")

Found 76 comments
Here's a sample:

Ozzie_osman 5 months ago  
             | next [–] 

LangChain is awesome. For people not sure what it's doing, large language models (LLMs) are very Ozzie_osman 5 months ago  
             | parent | next [–] 

Also, another library to check out is GPT Index (https://github.com/jerryjliu/gpt_index)


#2.Text Splitters 文件分割器



"块的大小"（chunk_size）：这个参数定义了每个分割出来的文本块的大小。这是因为语言模型处理文本时，通常有一个最大的输入长度限制。例如，OpenAI的GPT-3模型的最大输入长度是2048个token。如果你有一个非常长的文档，比如一本书，你不能直接将整本书的内容输入到模型中，因为这会超过模型的最大输入长度。因此，你需要将长文档分割成多个小块，每个小块的长度都不超过模型的最大输入长度。

"块的重叠"（chunk_overlap）：这个参数定义了相邻的两个文本块之间有多少字符是重叠的。这是因为当你将一个长文档分割成多个小块时，如果没有重叠，那么在分割点的地方可能会丢失一些上下文信息。例如，如果一个句子恰好在两个块的边界上，那么这个句子可能会被切断，导致模型无法理解这个句子的完整含义。通过设置一个合适的重叠长度，可以保证模型在处理每个小块时，都能获取到足够的上下文信息。

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('test.txt') as f:
    doc = f.read()

print (f"You have {len([doc])} document")

You have 1 document


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    #块的大小，一些LLM无法接受过大的文件块，需要对其进行切分
    chunk_size = 150,
    #块与块之间的重叠字符数，保持块与块之间上下文完整性
    chunk_overlap  = 20,
)

texts = text_splitter.create_documents([doc])
print (f"You have {len(texts)} documents")


You have 59 documents


#3.Retrievers 搜索器

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader('test.txt')
documents = loader.load()
# 准备文本分割器
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# 将大文档分成小块
texts = text_splitter.split_documents(documents)

#选择嵌入模型
embeddings = OpenAIEmbeddings(deployment='text-embedding-ada-002')
persist_directory = 'db'

#将文本保存到FAISS 向量数据库中
db = FAISS.from_documents(texts, embeddings)

# 准备文件搜索器
retriever = db.as_retriever()

docs = retriever.get_relevant_documents("文章的作者想表达什么思想？")
print("\n\n".join([x.page_content[:200] for x in docs[:2]]))



lot better than a typewriter.Though I liked programming, I didn't plan to study it in college.
In college I was going to study philosophy, which sounded much more
powerful. It seemed, to my naive high

computer called Mike, and a PBS documentary that showed Terry
Winograd using SHRDLU. I haven't tried rereading The Moon is a Harsh
Mistress, so I don't know how well it has aged, but when I read it
I 
